In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

<h1>Load the data</h1>
<p>Load the data</p>
<p>Seperate the target and the inputs</p>

In [2]:
data = pd.read_csv('gtd_utf.csv', encoding='latin1', low_memory=False)


# Seperate the target from the data

targetLabelEncoder = preprocessing.LabelEncoder()
targetLabelEncoder.fit(data['gname'].unique())
requiredtarget = targetLabelEncoder.transform(data['gname'])

requireduniquetarget = [data['gname'].unique(), list(range(len(data['gname'].unique())))]

<h1>Preprocessing of Data</h1>
<p>Remove any features that are not required<p>
<p>Set up data into the GTD object<p>

In [4]:
from sklearn.feature_extraction import DictVectorizer
features = data[['iyear', 'country_txt', 'region_txt', 'attacktype1_txt', 'targtype1_txt','weaptype1_txt' ,'extended', 
                 'specificity','vicinity', 'crit1','crit2','crit3', 'multiple', 'success', 'suicide' , 'attacktype2_txt',
                 'attacktype3_txt','targsubtype1_txt', 'corp1','target1', 'natlty1_txt', 'targtype2_txt', 'targsubtype2_txt', 
                 'corp2', 'target2','natlty2_txt']]
# features = data[['iyear', 'country_txt', 'region_txt', 'attacktype1_txt', 'targtype1_txt', 'extended', 'specificity', 'vicinity', 
#                 'crit1','crit2','crit3','doubtterr', 'alternative', 'multiple', 'success', 'suicide', 'attacktype2_txt', 'attacktype3_txt', 
#                 'targsubtype1_txt', 'corp1','target1', 'natlty1_txt', 'targtype2_txt', 'targsubtype2_txt', 'corp2', 'target2','natlty2_txt',
#                  'targtype3_txt', 'targsubtype3_txt', 'corp3', 'target3', 'natlty3', 'natlty3_txt', 'guncertain1', 'guncertain2', 'guncertain3', 
#                  'nperps', 'nperpcap','claimed','claimmode_txt', 'claim2', 'claimmode2_txt', 'claim3','claimmode3_txt', 'compclaim', 'weaptype1_txt',
#                 'weapsubtype1_txt','weaptype2_txt','weapsubtype2_txt', 'weaptype3_txt']] 
# , 'suicide'


df = (pd.DataFrame(features)).to_dict(orient='records')
dv = DictVectorizer(sparse=False) 
output = dv.fit_transform(features.to_dict(orient='records'))

print(output)

SyntaxError: invalid character in identifier (<ipython-input-4-2c969d1431c1>, line 4)

In [ ]:
np.shape(output)

<h1>Apply PCA</h1>
<p> Reduce dimensionality of the features
</p>

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=275)
reducedFeatures = pca.fit_transform(output)

<h1>Random Forest</h1>
<p> Fit the data and target using Random forest classifier
</p>

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1, max_depth=4, min_samples_split=2)
scores = cross_val_score(clf, reducedFeatures, requiredtarget)
scores.mean() 